# Time mean ocean Dynamic Topography

Ruth Moorman April 25th 2024 (for Ian Fenty ESE144 lectures)


In this notebook we'll be looking at some Level 4 processed mean dynamic topography data available from  <a href="https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_MDT_008_063/description?view=-&option=-&task=results&product_id=-">Copernicus Marine Service</a> and constructed by CNES (French National Center for Space Studies) using data spanning 1993-2012. The "Mean Dynamic Topography" (MDT) is the temporal mean of the sea surface heigh (SSH) above the geoid. We'll be using this data to understand the time mean geostrophic ciculation of the Earth's oceans. 
<br><br>



In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs


<div class="alert alert-block alert-info">
<b>XARRAY:</b>
<div> 
<div>        
  
Since netCDF data structures are now incredibly widespread in rsatellite remote sensing and climate science more broadly, we'll be using the <a href="https://docs.xarray.dev/en/stable/">xarray</a> python package, a suite of functions built to interface smoothly with netCDF files, for our python work in this course.
    
    <br><br>
In this notebook we'll use the following xarray functions:
   <br> 
    
    • <a href="https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html">xr.open_dataset()</a>: opens a singular netCDF file as a data structure called a Dataset (ds) containing numerous <a href="https://docs.xarray.dev/en/latest/user-guide/data-structures.html">DataArrays (da)</a>
   <br> 
    • <a href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.coarsen.html">da.coarsen()</a>: coarsens data in some dimension
<div>   
    

In [ ]:
ds = xr.open_mfdataset('~/shared/notebooks/CaltechESE1442024/data/mdt_hybrid_cnes_cls22_cmems2020_global.nc').squeeze("time")
ds


<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>        
  
Explore the variables available in this Dataset, familiarise yourself with the data structure if this is the first time you have used xarray.   
<div>   
    

## Extract variables for plotting

In [ ]:
mdt = ds["mdt"] # mean dynamic topography
u   = ds["u"]   # geostrophic zonal (eastward) velocity
v   = ds["v"]   # geostrophic meridional (northward) velocity

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,7),subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})

c = ax.contourf(mdt.longitude, mdt.latitude, mdt, np.arange(-1.5,1.95,0.1),transform = ccrs.PlateCarree(), cmap='nipy_spectral')

w=30 
coarse_u = u.coarsen(longitude=w,latitude=w, boundary='pad').mean()
coarse_v = v.coarsen(longitude=w,latitude=w, boundary='pad').mean()

lon, lat = np.meshgrid(coarse_u.longitude,coarse_u.latitude)
q = ax.quiver(x=lon,y=lat , u=coarse_u.values, v=coarse_v.values, transform = ccrs.PlateCarree())
ax.quiverkey(q, 0.1, 0.05, 0.25,'velocity (0.25 m s$^{-1}$)',labelpos='E',transform=ccrs.PlateCarree(),color='r')

ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.07, 0.2, 0.015, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)

ax.set_title('HYBRID_MDT_CNES_CLS22_CMEMS2020 - MDT [m] and geostrophic velocity [m/s]')

plt.show()



<div class="alert alert-block alert-info">
<b>DISCUSS:</b>
<div> 
<div>        
  Geostrophic velocities $u$ and $v$ are defined according to
    
    $$u = -\frac{1}{\rho f}\frac{\partial p}{\partial y}, \quad \quad v = \frac{1}{\rho f}\frac{\partial p}{\partial x}$$

<div>
    in terms of pressure ($p$). Or (with some assumptions...) the barotropic component of the geostrophic flow can be expressed as a function of the sea surface height anomaly ($\eta$)
    $$u = -\frac{g}{f}\frac{\partial \eta}{\partial y}, \quad \quad v = \frac{g}{f}\frac{\partial \eta}{\partial x}.$$

<div> 

Does this hold everywhere on Earth? If not, why/how does this product provide geostrophic $u$ and $v$ values for all latitudes and longitudes over the ocean?
    

    
   Let's explore <a href="https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_MDT_008_063/description?view=-&option=-&task=results&product_id=-">the data source</a> info to try and figure this out. 
<div>   
    

## Latest daily SLA Altimetry

Moving onto some recent data  <a href="https://data.marine.copernicus.eu/product/SEALEVEL_GLO_PHY_L4_NRT_008_046/description">here</a>! 


In [ ]:
ds = xr.open_mfdataset('~/shared/notebooks/CaltechESE1442024/data/nrt_global_allsat_phy_l4_20240425_20240425.nc').squeeze("time")
ds

In [ ]:
sla = ds["sla"]
u = ds["ugos"]
v = ds["vgos"]

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(15,7),subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)})

## colors
c = ax.pcolormesh(sla.longitude, sla.latitude, sla, vmin=-0.5,vmax=0.5,transform = ccrs.PlateCarree(), cmap='RdBu_r')

## colorbar
ax.set_xticks(np.arange(-180,190,30));ax.set_yticks(np.arange(-80,90,20))
cax = fig.add_axes([0.07, 0.2, 0.015, 0.6])
cbar=plt.colorbar(c,cax = cax,orientation='vertical',shrink = 0.5)

## title
ax.set_title(' Sea Surface Height measured by Altimetry and derived variables [m]')
plt.show()


<div class="alert alert-block alert-info">
<b>TASK:</b>
<div> 
<div>        
Plot a zoomed in version of the above focussed on the Gulf Stream and including velocity vectors.
    <div>   
    